In [24]:
!nvidia-smi  # Confirm GPU is active

Wed Apr 16 00:45:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   75C    P0             41W /   70W |    7124MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Evidence-Based Policy Scoring with Open LLMs (Zero-Shot)

This notebook lets you score policy documents using an open-source LLM and a rubric-based prompt. It uses Mistral-7B via Hugging Face. Steps include:
* Load Mistral-7B-Instruct via Hugging Face
* Accept .txt policy files in a folder
* Prompt the model using the evidence-based policy (EBP) rubric
* Parse the response and output a .csv with scores and justifications for them

> Go to Runtime > Change runtime type > GPU in Colab if not doing it locally.


In [19]:
!pip install -q transformers accelerate bitsandbytes sentencepiece
!pip install -q unstructured
!pip install bitsandbytes --prefer-binary --no-cache-dir --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 263.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 254.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 236.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 230.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 221.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 206.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 329.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 208.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 227.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 247.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 245.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import json
import re

# Handler for bad model text/need to only parse valid json blocks
def extract_first_json(text):
    matches = re.findall(r"\{(?:[^{}]|(?:\{.*?\}))*\}", text, re.DOTALL)
    for match in matches:
        try:
            return json.loads(match)
        except json.JSONDecodeError:
            continue
    print("No valid JSON block found.")
    return None

### Loading in models
Note:  Mistral is a huge model and will take a long time to cache initially. It will also take a long time to process docs through it. A smaller model, like Llama-3b, is easier to use locally via CPU. Use a GPU (e.g., Colab) to run Mistral.

In [2]:
from huggingface_hub import login
login()

In [31]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

model_id = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_id, token=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    load_in_4bit=True
)

llm = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=2048)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


In [32]:
rubric_prompt = """
Use the rubric below to score the policy document on five dimensions of evidence-based policymaking.
Each dimension should receive a score from 0 to 3, where:

0 = No evidence of the dimension
1 = Minimal or implicit evidence
2 = Moderate or partial evidence
3 = Clear, strong, and explicit evidence

### Dimensions:

1. Use of Empirical Research
- 0: No references to empirical evidence or data
- 1: Vague or anecdotal references (e.g., “studies show”)
- 2: Clear empirical support, but limited sourcing
- 3: Multiple, clearly cited, high-quality sources (e.g., peer-reviewed, systematic reviews)

2. Formal Evidence-Gathering Process
- 0: No structured data gathering
- 1: Informal or anecdotal input
- 2: Basic assessments (e.g., internal reports, cost estimates)
- 3: Formal tools (e.g., RCTs, modeling, pilot programs)

3. Transparency and Accessibility
- 0: No documentation or rationale
- 1: Minimal or internal-only documentation
- 2: Public access with basic explanation
- 3: Fully open access, replicable, with detailed methods

4. Expert and Stakeholder Input
- 0: No input from external parties
- 1: Informal or minimal consultation
- 2: Some formal consultation or review by stakeholders
- 3: Broad engagement with named experts or stakeholders

5. Evaluation and Iteration
- 0: No plan or evidence of evaluation
- 1: A vague or one-time evaluation mention
- 2: Evaluation included with some details
- 3: Robust plan or evidence of ongoing evaluation and iteration

### Note:
You will be asked to return a JSON object based on a policy document. Each dimension must include:
- A `score` (0–3)
- A `justification` with a full sentence explaining the score

Do not reuse this rubric text or copy examples into your answer.
"""

In [33]:
import os

folder_path = "/content/txt"
os.makedirs(folder_path, exist_ok=True)

print(f"Upload your .txt policy documents into: {folder_path}")


Upload your .txt policy documents into: /content/txt


In [34]:
os.getcwd()

'/content'

In [ ]:
from pathlib import Path
import os
import pandas as pd
import json
import re
from transformers import pipeline

# Set your batch folder here
BATCH_FOLDER = "txt_batches/batch_01"  # Change this to batch_02, batch_03, etc.
txt_folder = Path(BATCH_FOLDER)
results = []

# Define prompt rubric (keep the existing rubric_prompt from previous cell)
for filename in os.listdir(txt_folder):
    if not filename.endswith(".txt"):
        continue
    print(f"Processing {filename}")
    with open(txt_folder / filename, "r", encoding="utf-8") as f:
        doc_text = f.read()

    # Final prompt structure with no static JSON example, only dynamic instruction
    full_prompt = (
        "You are a policy analyst evaluating how evidence-based a policy document is.\n"
        "Below is a policy document. Read it carefully and then apply the rubric that follows.\n\n"
        + doc_text[:3500] + "\n\n"
        + "Now use this rubric to evaluate:\n"
        + rubric_prompt + "\n\n"
        + "Return only a JSON object. Each key should be a rubric dimension. Each value should be an object "
          "with a 'score' (0–3) and a 'justification' — a sentence explaining why that score was given based on the document.\n"
        + "Do not reuse examples or include placeholder text like '...'. Be specific, original, and concise in your justifications."
    )

    response = llm(full_prompt)[0]["generated_text"]
    print(f"Raw model output for {filename} (first 1000 chars):\n", response[:1000])

    parsed = extract_first_json(response)
    if not parsed:
        print(f"Could not extract valid JSON for {filename}")
        continue

    flat = {"filename": filename}
    for k, v in parsed.items():
        if isinstance(v, dict):
            flat[f"{k} Score"] = v.get("score")
            flat[f"{k} Justification"] = v.get("justification")
        else:
            flat[f"{k} Score"] = v
            flat[f"{k} Justification"] = ""
    results.append(flat)

if results:
    df = pd.DataFrame(results)
    df.to_csv("evidence_scores.csv", index=False)
    print("CSV saved as 'evidence_scores.csv'")
else:
    print("No valid outputs parsed.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🔍 Processing 2025-03-10_IN12522_d12b023987361bb7fa1d58c5c2013f97dbec1abc.txt


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🧠 Raw model output for 2025-03-10_IN12522_d12b023987361bb7fa1d58c5c2013f97dbec1abc.txt (first 1000 chars):
 You are a policy analyst evaluating how evidence-based a policy document is.
Below is a policy document. Read it carefully and then apply the rubric that follows.

 INSIGHTi 

An Overview of S. 394, GENIUS Act of 2025 

March 10, 2025 

On February 4, 2025, Senator Bill Hagerty introduced S. 394, the Guiding and Establishing National 
Innovation for U.S. Stablecoins Act of 2025, or GENIUS Act, which would establish a regime to regulate 
stablecoins. Currently, existing state and federal laws and regulations are applied to aspects of the 
stablecoin industry based on the nature of activities and individual stablecoin features.  

The bill’s proposed regulatory framework and licensing process are described below. 

Requirements for Issuing Payment Stablecoins  
S. 394 defines payment stablecoins as digital assets, issued for payment or settlement and redeemable at a 
predetermined 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

🧠 Raw model output for 2023-12-04_R45122_c45638a9e865aa13619c2c320667c61bf205aaa6.txt (first 1000 chars):
 You are a policy analyst evaluating how evidence-based a policy document is.
Below is a policy document. Read it carefully and then apply the rubric that follows.

Afghanistan: Background and U.S. Policy 

Updated December 4, 2023 

Congressional Research Service 

https://crsreports.congress.gov 

R45122 

 
 
  
 

Afghanistan: Background and U.S. Policy 

Over two years after the Taliban’s 2021 return to power, U.S. policymakers are still grappling 
with the reality of the group’s autocratic rule and the negative consequences that rule has had for 
many Afghans and U.S. policy interests. In 2021, U.S. and international forces withdrew from 
Afghanistan after nearly two decades and the Taliban, a Sunni Islamist extremist group that 
formerly ruled the country from 1996 to 2001, retook power. The United States does not 
recognize the Taliban or any other entity as the government

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🧠 Raw model output for 2025-03-07_IF10908_42fc60a760824c3944a140abbc85ddd13f56a5bb.txt (first 1000 chars):
 You are a policy analyst evaluating how evidence-based a policy document is.
Below is a policy document. Read it carefully and then apply the rubric that follows.

Costa Rica: An Overview

Costa Rica historically has been a bastion of stability in 
Central America and often has served as a key U.S. partner 
for advancing policy goals in the region. The 118th 
Congress sought to reinforce U.S.-Costa Rican security 
cooperation by appropriating security assistance for the 
country and authorizing the U.S. Armed Forces to engage 
in certain training with Costa Rican security forces. The 
119th Congress may monitor U.S.-Costa Rican efforts to 
address shared challenges, such as transnational crime and 
unauthorized migration, as well as potential opportunities, 
such as collaboration on supply chains in strategic sectors. 

Political Situation 
Costa Rica has sustained civilian democ

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🧠 Raw model output for 20190501_R45122_8dd78b3efd06e20151ecec6d9f8a9267f64a7825.txt (first 1000 chars):
 You are a policy analyst evaluating how evidence-based a policy document is.
Below is a policy document. Read it carefully and then apply the rubric that follows.

Afghanistan: Background and U.S. Policy In 
Brief 

Clayton Thomas 
Analyst in Middle Eastern Affairs 

Updated May 1, 2019 

Congressional Research Service 

7-....

www.crs.gov 

R45122 

 
 
  
 
 

Afghanistan: Background and U.S. Policy In 
Brief 

Afghanistan has been a central U.S. foreign policy concern since 2001, when the United States, 
in response to the terrorist attacks of September 11, 2001, led a military campaign against Al 
Qaeda and the Taliban government that harbored and supported it. In the intervening 17 years, 
the United States has suffered around 2,400 fatalities in Afghanistan (including seven in 2019 to 
date) and Congress has appropriated approximately $133 billion for reconstruction there. I